# Project 1
### Epidemiological Study: US Vaccination Campaign (November 2020 - March 2021)
---
### Project Description/Outline
Determine the effectiveness of the US vaccination campaign in attending the population that is most affected by covid-19 disease. This is achieved by revising the total number of doses applied, vaccination coverage, population demographics such as gender, race, socioeconomic status, and education, versus epidemiologic variables: incidence, prevalence, hospitalization, UCI, death numbers.

In [ ]:
## Dependencies
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt

# Import API key
from app_tokens import cdc_token

### Covid-19 Vaccination Data

In [ ]:
## Import data from CSV
hesitancy_df = pd.read_csv('data/Vaccine_Hesitancy_Covid19.csv')#, encoding='latin-1')
hesitancy_df.head()

### Pendiente
- Retrieve lat,lng as columns by fips
- "POINT (-86.844516 32.756889)	"
- separate


In [ ]:
columns = [
    'FIPS Code',
    'Social Vulnerability Index (SVI)',
    'SVI Category',
    'Percent adults fully vaccinated against COVID-19',
    'Percent Hispanic',
    'Percent non-Hispanic American Indian/Alaska Native',
    'Percent non-Hispanic Asian',
    'Percent non-Hispanic Black',
    'Percent non-Hispanic Native Hawaiian/Pacific Islander',
    'Percent non-Hispanic White'
]

vaccination_df = hesitancy_df[columns].sort_values('FIPS Code')
vaccination_df.reset_index(inplace=True, drop=True)

vaccination_df['Percent non-Hispanic Other'] = 1 - vaccination_df.iloc[:,4:10].sum(axis=1)
vaccination_df.head()

# del hesitancy_df

In [ ]:
vaccination_df.sort_values('Percent adults fully vaccinated against COVID-19', ascending=False).head(20)

In [ ]:
vaccination_df.shape

In [ ]:
x_values = vaccination_df['Social Vulnerability Index (SVI)']*100
y_values = vaccination_df['Percent adults fully vaccinated against COVID-19']*100
plt.scatter(x_values, y_values)
plt.show()

In [ ]:
x_values = vaccination_df.sort_values('Social Vulnerability Index (SVI)', ascending=False)['Social Vulnerability Index (SVI)'].head(300)*100
y_values = vaccination_df.sort_values('Social Vulnerability Index (SVI)', ascending=False)['Percent adults fully vaccinated against COVID-19'].head(300)*100

plt.scatter(x_values, y_values)

plt.xlabel("Vulnerability")
plt.ylabel("Vaccination")

plt.show()

### US Census Reference (2019)

In [ ]:
census_df = pd.read_csv('data/US_Census2019_totals.csv')#, encoding='latin-1')
census_df.head()

In [ ]:
census_2019 = census_df[['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'POPESTIMATE2019']]
census_2019.head()

In [ ]:
state_2019 = census_2019.loc[census_2019['COUNTY']==0]
state_2019.reset_index(inplace=True, drop=True)
# state_2019['POPESTIMATE2019'].sum()
state_2019

In [ ]:
county_2019 = census_2019.drop(census_2019.index[census_2019["COUNTY"]==0])
county_2019

In [ ]:
fips = []
for index, row in  county_2019.iterrows():
    fips.append(f'{row["STATE"]:>02}{row["COUNTY"]:>03}')
    
county_2019.insert(0, 'FIPS Code', fips)
county_2019.drop(labels=['STATE', 'COUNTY'], axis=1, inplace=True)

In [ ]:
county_2019.head()

In [ ]:
county_2019.shape

### Pendiente
- Agregar Male/Female
- Agregar AgeGroup
- Mover Ethnicity (desde Vaccination) en número / % ???


### Nomenclatura AGEGRP de US Census a age_group


In [ ]:
import numpy as np

all_data = pd.read_csv("data/cc-est2019-alldata.csv", encoding='latin-1')
all_data.head()

In [ ]:
age_conditions = [
    (all_data['AGEGRP'] == 0),
    (all_data['AGEGRP'] >= 1) & (all_data['AGEGRP'] <= 4),
    (all_data['AGEGRP'] >= 5) & (all_data['AGEGRP'] <= 10),
    (all_data['AGEGRP'] >= 11) & (all_data['AGEGRP'] <= 13),
    (all_data['AGEGRP'] >= 14) & (all_data['AGEGRP'] <= 18) 
]

age_values = ['0', '0 - 17 years', '18 - 49 years', '50 - 64 years', '65 + years']

In [ ]:
all_data['Age_group'] = np.select(age_conditions, age_values)
all_data.head()

### Covid-19 Case Surveillance


In [ ]:
months = ['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
          '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
          '2021-01', '2021-02', '2021-03']

fields = 'case_month, county_fips_code, current_status, sex, age_group, race, ethnicity, hosp_yn, icu_yn, death_yn'

fips_index = fips
fips_index.append('NA')

patients_df = pd.DataFrame(index=fips)
hospitalized_df = pd.DataFrame(index=fips)
icu_df = pd.DataFrame(index=fips)
death_df = pd.DataFrame(index=fips)

In [ ]:
fips_index[-1]
len(fips_index)

In [ ]:
query_url = "https://data.cdc.gov/resource/n8mc-b4w4.json?"
params = {
    '$$app_token': cdc_token,
    '$limit': 25000000,
    '$offset': 0,
    '$select': fields
}

In [ ]:
months = ['2020-01', '2020-02', '2020-03', '2020-04']

## Print Log Header
print("Beginning Data Retrieval")
print("------------------------------")

for month in months:
    
    ## Print Log Status
    print(f"Processing Month = {month} [{datetime.datetime.now().strftime('%H:%M:%S')}]")
    #print(f"Query {i+1}: {requests.get(query_url, params=params)} [{datetime.datetime.now().strftime('%H:%M:%S')}]")
        
    params['case_month'] = month
    response_month = requests.get(query_url, params=params).json()

    response_df = pd.DataFrame(response_month)

    #patients_df[month] = response_df[['county_fips_code', 'case_month']].groupby('county_fips_code').count()
    patients_df[month] = response_df.groupby('county_fips_code')['case_month'].count()
    hospitalized_df[month] = response_df.loc[response_df['hosp_yn'] == 'Yes'].groupby('county_fips_code')['death_yn'].count()
    icu_df[month] = response_df.loc[response_df['icu_yn'] == 'Yes'].groupby('county_fips_code')['icu_yn'].count()
    death_df[month] = response_df.loc[response_df['death_yn'] == 'Yes'].groupby('county_fips_code')['death_yn'].count()

    if month != months[-1]:
        print("Sleeping...")
        time.sleep(60*1)

## Print Log Footer
print("------------------------------")        
print("Data Retrieval Complete")
print("------------------------------")    

In [ ]:
patients_df.tail()

In [ ]:
hospitalized_df.tail()

In [ ]:
icu_df.tail()

In [ ]:
death_df.tail()

### Pendientes

1. Un DF con 18 meses (columnas) por: Deidentified Patients, Hosp_YN, ICU_YN, Death_YN
2. Al final, agregar suma de todos los meses
3. Un DF con todos los totales [patients, hosp, ]
4. Un DF con todos los totales [patients, hosp, ] (por c/100k hab)
    (total*pop/100,000)
5. Agregar Columna  Vacunados en 4 y 5


### Gráficas

...Peores fips por definir

1. Vacunados vs. tiempo (total/estado) + Pacientes vs. tiempo
2. Línea de tiempo (peores fips) vacunados vs afectados
3. Caracterizar (sexo, edad, etnicidad) a la media de los peores fips vs media mejores fips
4. Peores/Mejores fips: Avance vacunación (stacked bars)
5. Scatter (pacientes, vacunados) por fips
    a. Pearson + LinRegress
6. Heatmap (vacunación vs. afectados)
7. Regresiones por sexo, edad, grupo étnico
8. Barras agrupadas por grupo étnico





### Respaldo Mariana

In [ ]:
import pandas as pd
import sqlalchemy
from sodapy import Socrata

socrata_domain = 'data.cdc.gov'
socrata_dataset_identifier = "n8mc-b4w4"

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata(socrata_domain, None)

#Get metadata
metadata = client.get_metadata(socrata_dataset_identifier)
[x['name'] for x in metadata['columns']]

In [ ]:
results = client.get(socrata_dataset_identifier,limit = 24441351,
                      #where = "current_status"=="Laboratory-confirmed case",
                     select="county_fips_code,case_month,current_status,sex,age_group,race,ethnicity,hosp_yn,icu_yn,death_yn"
                    )
                    
tryout_df = pd.DataFrame.from_records(results)

In [ ]:
tryout_df.head()

In [ ]:
tryout_df['current_status'].value_counts()

In [ ]:
url = https://data.cdc.gov/resource/n8mc-b4w4.json?case_month=2020-12&current_status=Laboratory-confirmed case